# Capstone Week 3: Toronto Segmentation and Clustering

In [1]:
import sys
import os
from IPython.core.interactiveshell import InteractiveShell
from platform import python_version
print(python_version())
InteractiveShell.ast_node_interactivity = "all"
print("My working directory:\n" + os.getcwd())

3.7.1
My working directory:
C:\Users\David Zhou


In [2]:
#!pip install geopy

In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('Libraries imported.')

Libraries imported.


<h2>PART ONE: Get the location Data</h2>

<ol>
    <li>Get Postal Code, Borough and Neighborhood information from Wikipedia</li>
    <li>Get Geo Coordinates from Data provided and PyPl</li>
</ol>

<hr>

### Step 0: Read table from Wikipedia

In [4]:
temp = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [5]:
temp[0].head()

,0,1,2
0,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


<h3>Step 1:</h3>

<ol>
    <li>Take the table we need from result</li>
</ol>

<hr>

In [6]:
df_PostCodeM = pd.DataFrame(temp[0])
df_PostCodeM.columns = df_PostCodeM.iloc[0]
df_PostCodeM.drop([0], inplace = True)
df_PostCodeM.reset_index(drop=True, inplace=True)
df_PostCodeM.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<h3>Step 2: drop rows with 'Not assigned' Borough</h3>

In [7]:
df_PostCodeM_adj = df_PostCodeM[df_PostCodeM['Borough'] != 'Not assigned']
df_PostCodeM_adj.reset_index(drop=True, inplace=True)
df_PostCodeM_adj.head(10)
df_PostCodeM_adj.shape

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


(210, 3)

<h3>Step 3: Replace the 'Not assigned' Neighborhood with the value of borough</h3>

In [8]:
for i in range(df_PostCodeM_adj.shape[0]):
    if df_PostCodeM_adj.iloc[i]['Neighborhood'] == 'Not assigned':
        df_PostCodeM_adj.iloc[i]['Neighborhood'] = df_PostCodeM_adj.iloc[i]['Borough']

In [9]:
(df_PostCodeM_adj['Neighborhood']=='Not assigned').unique()
df_PostCodeM_adj.head(10)

array([False])

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


<h3>Step 4: merge neighborhoods with the same postal code</h3>

In [10]:
#a function to create a dataframe with merged neighborhoods with the same postal code
def merge_neighbor(temp):
    a = str()
    for item in temp['Neighborhood']:
        a = a + item +', '
    result_list = [temp.iloc[0,0], temp.iloc[0,1], a[0:-2]]
    result_df = pd.DataFrame(result_list, index=['Postcode', 'Borough', 'Neighborhood']).T
    return result_df

In [11]:
#get all the postal code (no repetition)
Postcode = df_PostCodeM_adj['Postcode'].unique()

#merge neighborhoods for each postal code
for item in Postcode:
    temp = df_PostCodeM_adj[df_PostCodeM_adj['Postcode'] == item]
    #temp
    result_df = merge_neighbor(temp)
    df_PostCodeM_adj = df_PostCodeM_adj[df_PostCodeM_adj['Postcode'] != item]
    df_PostCodeM_adj = df_PostCodeM_adj.append(result_df, ignore_index=True, sort=False)
    #df_PostCodeM_adj.shape

In [12]:
#show 11 rows and shape
df_PostCodeM_adj.head(5)
df_PostCodeM_adj.shape

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


(103, 3)

### Step 5: get the geographical coordinates of the neighborhoods

In [13]:
#It seems pgeocode is a better and more reliable package
#!pip install pgeocode
from pgeocode import Nominatim

In [14]:
#use PyPl pgeocode package find geographical coordinates
nomi = Nominatim('CA')
Latitude = []
Longitude = []

for item in df_PostCodeM_adj['Postcode']:
    Geo_Coord = nomi.query_postal_code(item)
    Latitude.append(Geo_Coord.loc['latitude'])
    Longitude.append(Geo_Coord.loc['longitude'])

In [15]:
#put those geo-coordinates into the dataframe
Latitude_df = pd.DataFrame(Latitude)
Longitude_df = pd.DataFrame(Longitude)
df_PostCodeM_adj['Latitude'] = Latitude_df
df_PostCodeM_adj['Longitude'] = Longitude_df
df_PostCodeM_adj.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7223,-79.4504
4,M7A,Downtown Toronto,Queen's Park,43.6641,-79.3889


#### check to see if there is something missing

In [16]:
df_anynull = df_PostCodeM_adj.isnull()
df_anynull.describe()
df_anynull['Latitude'].value_counts()
df_anynull['Longitude'].value_counts()
df_PostCodeM_adj[df_anynull['Latitude'] == True]

,Postcode,Borough,Neighborhood,Latitude,Longitude
count,103,103,103,103,103
unique,1,1,1,2,2
top,False,False,False,False,False
freq,103,103,103,102,102


False    102
True       1
Name: Latitude, dtype: int64

False    102
True       1
Name: Longitude, dtype: int64

,Postcode,Borough,Neighborhood,Latitude,Longitude
76,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


### Since there is a row missing value, I will use the geo-coordinates from the csv file provided to get 'M7R' value

In [18]:
#read from the CSV file
#!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data
df = pd.read_csv('Geospatial_Coordinates.csv')
df[df['Postal Code'] == 'M7R']

,Postal Code,Latitude,Longitude
86,M7R,43.636966,-79.615819


In [19]:
#get the missing data from CSV file and insert them into the data we are going to use
df_PostCodeM_adj.iloc[76,3] = df[df['Postal Code'] == 'M7R'].iloc[0,1]
df_PostCodeM_adj.iloc[76,4] = df[df['Postal Code'] == 'M7R'].iloc[0,2]

In [20]:
#check again to make sure there is no null
df_anynull = df_PostCodeM_adj.isnull()
df_anynull.describe()
df_anynull['Latitude'].value_counts()
df_anynull['Longitude'].value_counts()
df_PostCodeM_adj[df_anynull['Latitude'] == True]

,Postcode,Borough,Neighborhood,Latitude,Longitude
count,103,103,103,103,103
unique,1,1,1,1,1
top,False,False,False,False,False
freq,103,103,103,103,103


False    103
Name: Latitude, dtype: int64

False    103
Name: Longitude, dtype: int64

,Postcode,Borough,Neighborhood,Latitude,Longitude


In [21]:
df_PostCodeM_adj.head(11)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7223,-79.4504
4,M7A,Downtown Toronto,Queen's Park,43.6641,-79.3889
5,M9A,Queen's Park,Queen's Park,43.6662,-79.5282
6,M1B,Scarborough,"Rouge, Malvern",43.8113,-79.1930
7,M3B,North York,Don Mills North,43.7450,-79.3590
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7063,-79.3094
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6572,-79.3783


In [22]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_PostCodeM_adj['Borough'].unique()),
        df_PostCodeM_adj.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


<h2>PART Two: Use Foursquare API and location information from above steps to analyze the data</h2>

<ol>
    <li>Get nearby venures</li>
    <li>Clustering</li>
</ol>

<hr>

### Get nearby venures using Foursquare API

In [23]:
#!pip install folium

In [24]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[df_PostCodeM_adj.iloc[0,3], df_PostCodeM_adj.iloc[0,4]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_PostCodeM_adj['Latitude'], df_PostCodeM_adj['Longitude'], df_PostCodeM_adj['Borough'], df_PostCodeM_adj['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Foursquare Credentials version

In [26]:
# @hidden_cell
CLIENT_ID = 'WS5UF05OLI5UNPNAH02JCAQ1JKV2JIDNWJ2ZKTPMRZYROBZW' # your Foursquare ID
CLIENT_SECRET = 'YR0FRHBSHZ3CEOX4DJHYKYS1LIWESWMQARN4WMVOED3PC0MJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        #print(len(venues_list))
    #print(len(venues_list),len(venues_list[1]),len(venues_list[0][0]))
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    print(nearby_venues.shape)
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=df_PostCodeM_adj['Neighborhood'],
                                   latitudes=df_PostCodeM_adj['Latitude'],
                                   longitudes=df_PostCodeM_adj['Longitude']
                                  )

(2280, 7)


In [29]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7545,-79.3300,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.7545,-79.3300,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
2,Parkwoods,43.7545,-79.3300,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.7276,-79.3148,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.7276,-79.3148,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [30]:
# for each Neighborhood, how many venue types do we have
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Downsview North, Wilson Heights",6,6,6,6,6,6
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,92,92,92,92,92,92


In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print('There are {} uniques Neighborhood.'.format(len(toronto_venues['Neighborhood'].unique())))

There are 263 uniques categories.
There are 99 uniques Neighborhood.


### Clustering

#### Clustering data preparation

In [32]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['NEIGHBORHOOD'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_onehot.shape

,NEIGHBORHOOD,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

(2280, 264)

In [33]:
# group and take a mean of the frequency of occurence
toronto_grouped = toronto_onehot.groupby('NEIGHBORHOOD').mean().reset_index()
toronto_grouped.head()
toronto_grouped.shape

,NEIGHBORHOOD,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.03,0.01,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.03,0.0,0.0,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.02,0.0,0.01,0.02,0.0,0.0,0.0,0.01,0.03,0.01,0.

(99, 264)

In [34]:
# a function return # of top venues in a row
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# a function to return a list of columns for table creation use later
def create_columns(num_top_venues, name):
    indicators = ['st', 'nd', 'rd']
    columns = [name]
    
    for i in range(num_top_venues):
        try:
            columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
        except:
            columns.append('{}th Most Common Venue'.format(i+1))
        
    return columns

In [35]:
num_top_venues = 10
name = 'NEIGHBORHOOD'

columns = create_columns(num_top_venues, name)

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted[name] = toronto_grouped['NEIGHBORHOOD']

for i in range(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[i, 1:] = return_most_common_venues(toronto_grouped.iloc[i, :], num_top_venues)

neighborhoods_venues_sorted.head()

,NEIGHBORHOOD,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Asian Restaurant,Gastropub,Burger Joint,Japanese Restaurant,American Restaurant,Restaurant
1,Agincourt,Latin American Restaurant,Breakfast Spot,Badminton Court,Skating Rink,Shanghai Restaurant,Yoga Studio,Falafel Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Yoga Studio,Farmers Market,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Sandwich Place,Fast Food Restaurant,Pharmacy,Beer Store,Hardware Store,Video Store,Caribbean Restaurant,Fried Chicken Joint
4,"Alderwood, Long Branch",Convenience Store,Pub,Sandwich Place,Coffee Shop,Gym,Pizza Place,Pharmacy,Cuban Restaurant,Doner Restaurant,Fish & Chips Shop


#### Clustering

In [37]:
# set number of clusters
def get_kmeans_labels(toronto_grouped, kclusters):
    
    toronto_grouped_clustering = toronto_grouped.drop('NEIGHBORHOOD', 1)

    # run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

    # check cluster labels generated for each row in the dataframe
    return kmeans.labels_

kclusters = 6
kmeans_labels_ = get_kmeans_labels(toronto_grouped, kclusters)
kmeans_labels_

array([0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 3, 3, 4, 3, 3, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,
       5, 0, 3, 0, 0, 0, 3, 0, 5, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 3, 1, 0, 0, 3])

In [38]:
# add clustering labels
def get_toronto_merged(kmeans_labels, df_PostCodeM_adj, neighborhoods_venues_sorted):
    
    temp = neighborhoods_venues_sorted.copy()
    temp.insert(0, 'Cluster Labels', kmeans_labels)
    toronto_merged = df_PostCodeM_adj

    # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
    toronto_merged = toronto_merged.join(temp.set_index('NEIGHBORHOOD'), on='Neighborhood')

    toronto_merged["Cluster Labels"].replace(np.nan, 0, inplace=True)
    toronto_merged[["Cluster Labels"]] = toronto_merged[["Cluster Labels"]].astype('int')
    
    return toronto_merged

toronto_merged = get_toronto_merged(kmeans_labels_, df_PostCodeM_adj, neighborhoods_venues_sorted)
toronto_merged.shape
toronto_merged.head()

(103, 16)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,3,Food & Drink Shop,Bus Stop,Park,Flower Shop,Fish Market,Fish & Chips Shop,Food,Field,Dumpling Restaurant,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.7276,-79.3148,0,Hockey Arena,Pizza Place,Portuguese Restaurant,French Restaurant,Park,Coffee Shop,Intersection,Event Space,Eastern European Restaurant,Electronics Store
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626,0,Coffee Shop,Restaurant,Breakfast Spot,Italian Restaurant,Mexican Restaurant,Spa,Food Truck,Event Space,Electronics Store,Beer Store
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7223,-79.4504,0,Clothing Store,Coffee Shop,Cosmetics Shop,Electronics Store,Jewelry Store,Men's Store,Toy / Game Store,Bakery,Food Court,Restaurant
4,M7A,Downtown Toronto,Queen's Park,43.6641,-79.3889,0,Coffee Shop,Gym,Italian Restaurant,Pharmacy,Park,Burrito Place,Portuguese Restaurant,Beer Bar,Sushi Restaurant,College Theater


In [39]:
#Check to see if there is NaN in 'Cluster Labels'
df_anynull = toronto_merged.isnull()
df_anynull.describe()
toronto_merged[df_anynull['Cluster Labels'] == True]

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
count,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103,103
unique,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,103,103,103,103,103,103,100,100,100,100,100,100,100,100,100,100


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [40]:
# create map
def create_map(df_PostCodeM_adj, kclusters,toronto_merged):
    map_clusters = folium.Map(location=[df_PostCodeM_adj.iloc[0,3], df_PostCodeM_adj.iloc[0,4]], zoom_start=11)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
    return map_clusters
    
map_clusters = create_map(df_PostCodeM_adj, kclusters,toronto_merged)
map_clusters

In [42]:
# define a function can do the whole process of clustering
kclusters = 6
def full_clustering(toronto_grouped, kclusters, df_PostCodeM_adj, neighborhoods_venues_sorted):
    kmeans_labels_ = get_kmeans_labels(toronto_grouped, kclusters)
    toronto_merged = get_toronto_merged(kmeans_labels_, df_PostCodeM_adj, neighborhoods_venues_sorted)
    map_clusters = create_map(df_PostCodeM_adj, kclusters,toronto_merged)
    
    return map_clusters

full_clust = full_clustering(toronto_grouped, kclusters, df_PostCodeM_adj, neighborhoods_venues_sorted)
full_clust

### We now can try different numbers of clustering to see which one seems good. By means of good, we can see a group with at least a certain number been differentiated.

In [77]:
for kclusters in range(2,20):
    print('Clustering ', kclusters)
    kmeans_labels_ = get_kmeans_labels(toronto_grouped, kclusters)
    toronto_merged = get_toronto_merged(kmeans_labels_, df_PostCodeM_adj, neighborhoods_venues_sorted)
    toronto_merged.groupby('Cluster Labels').count()
    full_clust = full_clustering(toronto_grouped, kclusters, df_PostCodeM_adj, neighborhoods_venues_sorted)
    full_clust   

Clustering  2


,Postcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,86,86,86,86,86,83,83,83,83,83,83,83,83,83,83
1,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17


Clustering  3


,Postcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,82,82,82,82,82,79,79,79,79,79,79,79,79,79,79
1,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19
2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


Clustering  4


...

,Postcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,60,60,60,60,60,57,57,57,57,57,57,57,57,57,57
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2


Clustering  18


,Postcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,8,8,8,8,8,5,5,5,5,5,5,5,5,5,5
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


Clustering  19


,Postcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,6,6,6,6,6,3,3,3,3,3,3,3,3,3,3
1,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
2,47,47,47,47,47,47,47,47,47,47,47,47,47,47,47
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


### From different maps, it seems that the one with 9 clusters is the one with most balanced clustering. We can use this one to dive into detailed information.

In [78]:
kclusters = 9
kmeans_labels_ = get_kmeans_labels(toronto_grouped, kclusters)
toronto_merged = get_toronto_merged(kmeans_labels_, df_PostCodeM_adj, neighborhoods_venues_sorted)
toronto_merged.head()
full_clust = full_clustering(toronto_grouped, kclusters, df_PostCodeM_adj, neighborhoods_venues_sorted)
full_clust

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.3300,0,Food & Drink Shop,Bus Stop,Park,Flower Shop,Fish Market,Fish & Chips Shop,Food,Field,Dumpling Restaurant,Fast Food Restaurant
1,M4A,North York,Victoria Village,43.7276,-79.3148,7,Hockey Arena,Pizza Place,Portuguese Restaurant,French Restaurant,Park,Coffee Shop,Intersection,Event Space,Eastern European Restaurant,Electronics Store
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626,5,Coffee Shop,Restaurant,Breakfast Spot,Italian Restaurant,Mexican Restaurant,Spa,Food Truck,Event Space,Electronics Store,Beer Store
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7223,-79.4504,5,Clothing Store,Coffee Shop,Cosmetics Shop,Electronics Store,Jewelry Store,Men's Store,Toy / Game Store,Bakery,Food Court,Restaurant
4,M7A,Downtown Toronto,Queen's Park,43.6641,-79.3889,5,Coffee Shop,Gym,Italian Restaurant,Pharmacy,Park,Burrito Place,Portuguese Restaurant,Beer Bar,Sushi Restaurant,College Theater


In [64]:
toronto_merged.groupby('Cluster Labels').count()

,Postcode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,,,
0,22,22,22,22,22,19,19,19,19,19,19,19,19,19,19
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,52,52,52,52,52,52,52,52,52,52,52,52,52,52,52
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23
8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


## Cluster in detail: 


### Since there is only 1 unit in cluster 1 - 4, 6, 8, we can focus only on 0, 5, 7.

In [74]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Food & Drink Shop,Bus Stop,Park,Flower Shop,Fish Market,Fish & Chips Shop,Food,Field,Dumpling Restaurant,Fast Food Restaurant
7,North York,0,Construction & Landscaping,Pool,Park,Basketball Court,Fish Market,Flower Shop,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
13,North York,0,Gym,River,Park,Yoga Studio,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Space
21,York,0,Park,Sporting Goods Shop,Beer Store,Bakery,Gym,Fast Food Restaurant,Market,Mexican Restaurant,Women's Store,Fish Market
27,North York,0,Park,Residential Building (Apartment / Condo),Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio,Donut Shop
35,East York,0,Convenience Store,Massage Studio,Greek Restaurant,Coffee Shop,Park,Falafel Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
36,Downtown Toronto,0,Park,Harbor / Marina,Music Venue,Café,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Yoga Studio,Dumpling Restaurant
39,North York,0,Construction & Landscaping,Trail,Park,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Donut Shop
46,North York,0,Park,Pool,Yoga Studio,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
61,Central Toronto,0,Park,Photography Studio,Yoga Studio,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market


In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,5,Coffee Shop,Restaurant,Breakfast Spot,Italian Restaurant,Mexican Restaurant,Spa,Food Truck,Event Space,Electronics Store,Beer Store
3,North York,5,Clothing Store,Coffee Shop,Cosmetics Shop,Electronics Store,Jewelry Store,Men's Store,Toy / Game Store,Bakery,Food Court,Restaurant
4,Downtown Toronto,5,Coffee Shop,Gym,Italian Restaurant,Pharmacy,Park,Burrito Place,Portuguese Restaurant,Beer Bar,Sushi Restaurant,College Theater
5,Queen's Park,5,Coffee Shop,Gym,Italian Restaurant,Pharmacy,Park,Burrito Place,Portuguese Restaurant,Beer Bar,Sushi Restaurant,College Theater
9,Downtown Toronto,5,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Theater,Fast Food Restaurant,Sporting Goods Shop,Restaurant,Ramen Restaurant
15,Downtown Toronto,5,Coffee Shop,Café,Restaurant,Hotel,Seafood Restaurant,Bakery,Clothing Store,Cocktail Bar,Breakfast Spot,Italian Restaurant
16,York,5,Playground,Trail,Grocery Store,Field,Deli / Bodega,Park,Coffee Shop,Tennis Court,Hockey Arena,Auto Dealership
19,East Toronto,5,Other Great Outdoors,Pub,Coffee Shop,Bakery,Cheese Shop,Trail,Neighborhood,Gastropub,Health Food Store,Ethiopian Restaurant
20,Downtown Toronto,5,Coffee Shop,Hotel,Café,Bakery,Restaurant,Cocktail Bar,Seafood Restaurant,Japanese Restaurant,Beer Bar,Farmers Market
23,East York,5,Sporting Goods Shop,Sports Bar,Electronics Store,Restaurant,Liquor Store,Bakery,Burger Joint,Sandwich Place,Breakfast Spot,Furniture / Home Store


In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,7,Hockey Arena,Pizza Place,Portuguese Restaurant,French Restaurant,Park,Coffee Shop,Intersection,Event Space,Eastern European Restaurant,Electronics Store
8,East York,7,Fast Food Restaurant,Pizza Place,Intersection,Bank,Gym / Fitness Center,Breakfast Spot,Gastropub,Pet Store,Pharmacy,Field
10,North York,7,Pizza Place,Fast Food Restaurant,Grocery Store,Gas Station,Latin American Restaurant,Japanese Restaurant,Mediterranean Restaurant,Hardware Store,Doner Restaurant,Fish & Chips Shop
11,Etobicoke,7,Pizza Place,Coffee Shop,Tea Room,Sandwich Place,Chinese Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Donut Shop
14,East York,7,Spa,Convenience Store,Beer Store,Asian Restaurant,Video Store,Farmers Market,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space
17,Etobicoke,7,Convenience Store,Beer Store,Pharmacy,Pet Store,Coffee Shop,Café,Shopping Plaza,Liquor Store,Pizza Place,Gym Pool
18,Scarborough,7,Pizza Place,Grocery Store,Coffee Shop,Video Game Store,Fast Food Restaurant,Beer Store,Discount Store,Supermarket,Fried Chicken Joint,Electronics Store
28,North York,7,Mediterranean Restaurant,Fried Chicken Joint,Deli / Bodega,Coffee Shop,Middle Eastern Restaurant,Pizza Place,Health Food Store,History Museum,Fish & Chips Shop,Field
34,North York,7,Sandwich Place,Massage Studio,Middle Eastern Restaurant,Pizza Place,Sports Bar,Fish Market,Fish & Chips Shop,Field,Dumpling Restaurant,Farmers Market
38,Scarborough,7,Coffee Shop,Intersection,Chinese Restaurant,Sandwich Place,Light Rail Station,Bus Station,Bank,Discount Store,Grocery Store,Bus Line


<h2>Conclusion: from above tables, we can roughly define:</h2>

<ol>
    <li>Cluster 0 are locations with parks and community spaces.</li>
    <li>Cluster 5 are locations with lots of resteraunts and street retails (downtown or centers).</li>
    <li>Cluster 7 are locations with resteraunts, super markets and other neighborhood retails (suburb centers).</li>
</ol>

<hr>